# 4. 챗봇엔진에 필요한 딥러닝 모델

## 4.1 keras

* 케라스는 신경망모델 구축시 필요한 라이브러리이다.
* pip install tensorflow

In [ ]:
# !pip install tensorflow
!pip show tensorflow

In [ ]:
# MNIST 분류모델 학습
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 데이터 즉, pixel값 0~255인 값을 0~1사이의 값을 정규화
X_train, X_test = X_train/255.0, X_test/255.0

In [ ]:
print(X_train.shape, X_test.shape)

# tf.data를 이용하여 데이터를 shuffling
ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000)
train_size = int(len(X_train) * 0.7) # 학습용 vs 검증용 = 7:3
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).batch(20)
print(type(train_size), type(train_ds), type(val_ds))
print(train_size)

In [ ]:
# MNIST분류모델
# keras 모델을 만드는 방법 순차모델, 함수형모델 2가지가 있다.
# 1. 모델정의
model = Sequential() # 순차모델
model.add(Flatten(input_shape=(28,28))) # 28x28 = 784의 1차원 입력데이터로 변환
model.add(Dense(20, activation='relu' ))
model.add(Dense(20, activation='relu' ))
model.add(Dense(10, activation='softmax'))

In [ ]:
# 2. 모델생성
# 오차계산함수 : sparse_categorical_crossentropy
# 오차옵티마이저 : sgd
# 측정항목 : accuracy 정확도로 설정
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
# 3. 모델학습
hist = model.fit(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
# 4. 모델평가
model.evaluate(X_test, y_test)
model.summary()

In [ ]:
# 5. 모델저장
model.save('./data/chatbot/mnist_model.keras')

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'D2Coding'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 6. 시각화
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()
loss_ax.plot(hist.history['loss'], 'y', label='학습용오류')
acc_ax.plot(hist.history['val_loss'], 'r', label='검증용오류')
acc_ax.plot(hist.history['accuracy'], 'y', label='학습용정확도')
loss_ax.plot(hist.history['val_accuracy'], 'r', label='학습용정확도')
loss_ax.plot(hist.history['loss'], 'y', label='학습용오류')
loss_ax.set_xlabel('훈련단계')
loss_ax.set_ylabel('정확도')
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
plt.show()

In [ ]:
# 학습된 모델을 사용해서 예측하기
from tensorflow.keras.models import load_model

_, (X_test, y_test) = mnist.load_data()
X_test = X_test / 255.0

model = load_model('./data/chatbot/mnist_model.keras')
model.summary()
model.evaluate(X_test, y_test, verbose=2)

In [ ]:
plt.imshow(X_test[100]) # , cmap='gray')
plt.show()

In [ ]:
# 20번째 이미지 분류하기
import numpy as np
img = [100]
predict = np.argmax(model.predict(X_test[img]))
print(f'손글씨 이미지의 예측한 숫자 = {predict}')

### 4.2.1 CNN 모델

* CNN모델은 이미지, Computer Vision 등의 판단에 특화되어 있는 알고리즘
* CNN알고리즘으로 `감정분류`도 가능하다.
* CNN은 합성곱(Convolution), 풀링(Pooling)연산으로 처리되는 알고리즘
* 합성곱은 필터(filter)라고 하는 특정크기의 행렬 소스(이미지행렬, 문장행렬...)와 곱하거나 더하는 연산을 수행
* 합성곱필터는 경우에 따라 `마스크 mask, 윈도우 window, 커널 kernel`등으로 다양하게 호칭되지만 보통 `필터 or 커널`로 호칭한다.
* 필터의 위치를 몇 칸씩 이동할지를 결정하는 값이 `스트라이드 stride`라고 한다.
* 연산을 거칠 때 마다 특징맵(연산후 결과를 feature map이라고 한다)의 크기가 작아 지는데 이를 방지하기 위해 `패딩 padding`을 사용한다.
* `패딩과 스트라드에 따라 출력크기는 keras가 자동으로 계산`해주기 때문에 개념만 알고 있으면 된다.

### 4.2.2 CNN모델로 문장데이터의 감정 분류 모델 구현하기

* chatbot_data.csv의 구조
|Q|A|Label|
|:------|:----------|:----:|
|12시 땡!|하루가 또 가네요.|0|
|1지망 학교 떨어졌어|위로해 드립니다.|0|
|3박4일 놀러가고 싶다|여행은 언제나 좋죠.|0|
|3박4일 정도 놀러가고 싶다|여행은 언제나 좋죠.|0|
|PPL 심하네|눈살이 찌푸려지죠.|0
|SD카드 망가졌어|다시 새로 사는 게 마음 편해요.|0|
|SD카드 안돼|다시 새로 사는 게 마음 편해요.|0|

* 분류
|Label|의미|
|:----:|:---------|
|0|일상다반사|
|1|이별(부정)|
|2|사랑(긍정)|


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [ ]:
# 1. 데이터읽어오기
train_file = './data/chatbot/chatbot_data.csv'
data = pd.read_csv(train_file)
data.tail()
features = data.Q.tolist()
labels = data.label.tolist() # or labels = list(data.label)
print(features[:5], labels[:5])
print(features[-5:], labels[-5:])

In [ ]:
# 2. 단어인덱스시퀀스벡터생성
# 질문리스트(features)에서 문장(text)을 하나씩 꺼내서 단어들의 시퀀스를 생성
# preprocessing.text.text_to_word_sequence(text)함수를 사용
corphs = [preprocessing.text.text_to_word_sequence(text) for text in features]
print(type(corphs), corphs[:5], '\총건수=', len(corphs))

In [ ]:
# 3. 토크나이징 & 단어시퀀스
# preprocessing.text.Tokenizer()
tokenizer = preprocessing.text.Tokenizer()
print(type(tokenizer))
tokenizer.fit_on_texts(corphs) # 기계학습

sequences = tokenizer.texts_to_sequences(corphs)
print(type(sequences), sequences[:5])

word_index = tokenizer.word_index
print(type(word_index), word_index['너무'])

# 시퀀스번호로 생성된 벡터의 크기가 제각각이기 때문에 CNN의 입력계층에 고정된 고정된 크기의 
# 입력데이터 크기를 설정
MAX_SEQ_LEN = 15

# 고정된 크기보다 작은 벡터에 남은 공간을 0로 채우는 padding처리
# maxlen을 너무 크게 설정하면 메모리낭비발생, 너무 작게 설정하면 데이터손실이 발생
# padding : pre-데이터앞에 패딩처리, post-데이터뒤에 패딩처리 옵션
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')
print(type(padded_seqs), len(padded_seqs))
padded_seqs

In [ ]:
# 4. 학습용, 검증용, 테스트요 데이터셋(7:2:1)
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
print(type(ds), ds)

train_size = int(len(features) * .7)
val_size = int(len(features) * .2)
test_size = int(len(features) * .1)
print(len(features), train_size, val_size, test_size, train_size+val_size+test_size)

train_ds = ds.take(train_size).batch(20) 
val_ds = ds.take(val_size).batch(20) 
test_ds = ds.take(test_size).batch(20) 

print(type(train_ds), train_ds)

In [ ]:
# 5. CNN모델을 정의
# 1) hyper parameter(옵션)을 정의
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1

# 2) CNN모델정의 - 케라스 함수형 모델방식으로 정의
#    전처리된 입력데이터를 단어 임베딩처리영역과 연산을 통해 문장의 특정배을 추출하고
#    평탄화를 거쳐서 완전 연결계츠 fully connection layer를 통해 감정별로 분류하는
#    영역으로 구성

# a. 입력계층
input_layer = Input(shape=(MAX_SEQ_LEN, )) # [ 4646,  4647,     0, ...,     0,     0,     0]
print(type(input_layer), input_layer)

In [ ]:
# b. 임베딩계층
#    단어별로 패딩처리된 시퀀스벡터(희소벡터)를 입력받아서 데이터손실을 최소화하면서
#    벡터차원이 압축되는 밀집벡터로 변환작업
#    단어의 갯수(VOCAB_SIZE)와 임베딩결과, 출력되는 밀집벡터의크기(EMB_SIZE), 입력되는
#    시퀀스벡터의 크기(MAX_SEQ_LEN)를 기준으로 임베딩계층을 생성
embedding_layer = Embedding(input_dim=VOCAB_SIZE
                          , output_dim=EMB_SIZE
                          , input_length=MAX_SEQ_LEN)(input_layer) # [ 4646,  4647,     0, ...,     0,     0,     0]

In [ ]:
# c. dropout계층
#    과적합화를 방지하기 위해 입력되는 데이터의 50%만 다음 layer의 입력데이터로 전달
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer) # [ 4646,  4647,     0, ...,     0,     0,     0] -> 50%만 전달

In [ ]:
# d. 임베딩벡터의 특징을 추출하는 계층
#    임베딩계층에서 전달된 벡터의 특징을 추출
#    커널의 크기가 (3, 4, 5)인 합성곱필터를 통해 MaxPooling의 결과를 다음계층으로 전달
conv1 = Conv1D(128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)  # 커널 = 3
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)  
pool2 = GlobalMaxPool1D()(conv1)
conv3 = Conv1D(128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)  
pool3 = GlobalMaxPool1D()(conv1)

In [ ]:
# e. n-gram(3,4,5) 이후 conv, pool계층을 병합
#    특징맵의 결과를 하나로 합치기
concat = concatenate([pool1, pool2, pool3])
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)

In [ ]:
# f. softmax
#    최종적으로 다중분류를 softmax함수를 통해서 감정별((0,1,2)확률계산
# predictions = Dense(3, activation=tf.nn.softmax)(logits) # 에러발생으로 보류, 나중에 확인할 것
predictions = Dense(3, activation='softmax')(logits)

In [ ]:
# g. 모델생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])         

In [ ]:
# h. 모델학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

In [ ]:
# i. 모델평가
loss, accuracy = model.evaluate(test_ds, verbose=1)
print(f'오차 = {loss:.3f}, 정확도 = {accuracy:.3f}')

In [ ]:
# j. 모델저장
model.save('./data/chatbot/cnn_model.keras')

### 4.2.3 훈련된 CNN모델을 이용

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

In [ ]:
# 1. 데이터읽기
train_file = './data/chatbot/chatbot_data.csv'
data = pd.read_csv(train_file)
features = data.Q.tolist()
lables = data.label.tolist()
print(features[:5])
print(labels[:5])

In [ ]:
# 2. 단어인덱스시퀀스벡터생성
corphs = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corphs)
sequences = tokenizer.texts_to_sequences(corphs)

MAX_SEQ_LEN = 15

padded_seqs = preprocessing.sequence.pad_sequences(sequences
                                                   , maxlen=MAX_SEQ_LEN
                                                   , padding='post')

In [ ]:
# 3. 테스트데이터셋생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20)

In [ ]:
# 4. 훈련된 CNN모델 불러오기
model = load_model('./data/chatbot/cnn_model.keras')
model.summary()

In [ ]:
# 5. 평가
loss, accuracy = model.evaluate(test_ds, verbose=1)
print(f'손실율 = {loss:.3f}, 정확도 = {accuracy:.3f}')

In [ ]:
# 6. 임의의 데이터로 학습된 모델로 예측하기
# 1) test데이터선택
sts = 11800
print(f'{sts}단어의 시퀀스 = {corphs[sts]}')
print(f'{sts}단어의 인덱스 시퀀스 = {padded_seqs[sts]}')
print(f'{sts}단어의 문장분류(정답) = {labels[sts]}')

In [ ]:
# 2) 예측
picks = [sts]
predict = model.predict(padded_seqs[picks])
print(type(predict), predict)
predict_class = tf.math.argmax(predict, axis=1)

In [ ]:
print(predict_class)
print(f'감정예측점수 = {predict}')
print(f'감정예측분류 = {predict_class}')
print(type(predict_class))
print(type(predict_class.numpy()), predict_class.numpy())

## 4.3 개체명인식을 위한 양방향 LSTM모델(BI-LSTM)

### 4.3.1 RNN(Recurrent Neural Network)

* I, google, at, work라는 문장이 주어졌을 때
  - I work at google. (work=동사, google=명사)
  - I google at work. (google=동사, work=명사)
  - work, google이란 단어는 각각 '명사, 동사'로 각각 쓰일 수가 있다.
* RNN은 시계열이나 텍스트데이터 분석에 용이하게 사용된다.
* LSTM은 RNN(Recurrent Neural Network)에서 파생된 모델
* RRN순환신경망으로 불리는데 은닉층노드의 출력값을 출력층과 다음의 은닉층노드의 입력으로 전달해서 순환하는 특징이 있다.
* 출력층 : y = wx = b
* 은닉층 : h = tanh(현재(y) + 과거(y_1) + b)
><img src="./images/0604.RNN_01.png" width="300" height="200" />
><img src="./images/0604.RNN_02.png" width="300" height="200" />
><img src="./images/0604.RNN_03.png" width="300" height="200" />
><img src="./images/0604.RNN_04.png" width="300" height="200" />
><img src="./images/0604.RNN_05.png" width="300" height="200" />
><img src="./images/0604.RNN_06.png" width="300" height="200" />
><img src="./images/0604.RNN_07.png" width="300" height="200" />
><img src="./images/0604.RNN_08.png" width="300" height="200" />
><img src="./images/0604.RNN_09.png" width="300" height="200" />
><img src="./images/0604.RNN_10.png" width="300" height="200" />
><img src="./images/0604.RNN_11.png" width="300" height="200" />

In [ ]:
# sin곡선
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, SimpleRNN, LSTM

In [ ]:
# time step만큼 시퀀스데이터를 생성(분리)함수
def split_sequence(sequence, step):
    X, y = list(), list()
    
    for i in range(len(sequence)):
        end_idx = i + step
        if end_idx > len(sequence) - 1:
            break
            
        seq_x, seq_y = sequence[i:end_idx], sequence[end_idx]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [ ]:
# 1. sin 학습데이터
# -10~10까지의 x축을 가지는 sin()함수의 값을 0.1씩 증가호 train_y에 저장
X = [i for i in np.arange(start=-10, stop=10, step=0.1)]
train_y = [np.sin(i) for i in X]
print(X[:5], '\n', train_y[:5])

In [ ]:
# 2. 하이퍼파라미터설정
# RNN의 입력시퀀스길이를 15로 정의 -> n_timesteps수 만큼 RNN셀을 생성
# 입력데이터의 차원(벡터) -> n_features = 1
n_timesteps = 15
n_features = 1

# 시퀀스나누기
# RRNN모델의 입력시퀀스를 나누기 위해 split_sequences()함수를 호출
# sin파형의 학습데이터 train_y에서 입력시퀀스길이 만큼 나눠서 입력시퀀스를 생성
X_train, y_train = split_sequence(train_y, step=n_timesteps)
print(f'shape X_train:{X_train.shape}, shape y_train:{y_train.shape}')

In [ ]:
# 3. RNN입력벡터크기를 맞추기 위해 벡터차원의 크기를 변경
#    keras.RRN계층을 사용하려면 3차원텐서형태이어야 한다.
#    따라서 2차원인 X_train의 차원을 RRN모델의 입력형태에 맞게 3차원(배치크기, 타임스템, 입력길이)
#    으로 변환 -> reshape함수를 이용
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], n_features)
print(f'shape X_train:{X_train.shape}, shape y_train:{y_train.shape}')

In [ ]:
# 4. RNN모델을 정의
#    sin파형의 데이터셋을 학습하기 위한 RRN계층을 정의한 후에 모델을 생성
#    모델 = SimpleRNN + Dense계층으로 구성
#    SimpleRNN은 가장 간단한 RNN계층
#    ... units=10 -> RNN계층에 존재하는 전체 뉴런의 갯수
#    ... return_sequences=False -> RRN계산과정에서 은닉상태값의 출력여부
#        -> False : 마지막 셀결과만 출력
#        -> True  : 모든 RNN계산과정의 결과를 출력
#        -> 이 옵션은 one-to-many, many-to-many구조를 위해 사용
#    ... input_shape=(n_timesteps, n_features) -> 입력데이터의 크기를 설정
model = Sequential()
model.add(SimpleRNN(units=10, return_sequences=False, input_shape=(n_timesteps, n_features)))
model.add(Dense(1))
# 손실함수=mse(정답과 오차의 차이), 옵티마이저(기울기조정)함수= adam
model.compile(optimizer='adam', loss='mse')

In [ ]:
# 5. 모델학습
# EarlyStopping콜백객체를 사용해서 손실의 변동이 없거나 갑자기 증가하는 시점에 조기중단
np.random.seed(42)
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=5, mode="auto")
history = model.fit(X_train, y_train, epochs=1000, callbacks=[early_stopping])

In [ ]:
# 6. loss시각화
plt.plot(history.history['loss'], label="loss")
plt.legend(loc='upper right')
plt.show()

In [ ]:
# 7. 학습된 RNN모델로 예측
# 1) 테스트데이터셋생성
#    ... 10~20사이의 x축범위를 가지는 cod()함수값을 0.1단위로 증가후에 y_calc에 저장
#    ... y_calc에 RNN모델을 테스트하기 위한 전체 시퀀스값을 저장
#    ... cos()함수를 이용하는 이유는 학습된 sin파형과 주기적인 차이를 전달하기 위해 임의로 생성
X_test = np.arange(10, 20, 0.1)
y_calc = np.cos(X_test)

# 2) RNN모델로 예측 및 로그를 저장
y_test = y_calc[:n_timesteps]

for i in range(len(X_test) - n_timesteps):
    net_input = y_test[i:i+n_timesteps]
    net_input = net_input.reshape((1, n_timesteps, n_features))
    y_train = model.predict(net_input, verbose=0)
    # print(y_test.shape, y_train.shape, i, i+n_timesteps)
    y_test = np.append(y_test, y_train)  

In [ ]:
# 3) 예측결과 시각화
plt.plot(X_test, y_calc, label="Real Value", color="orange")
plt.plot(X_test, y_test, label="Prediction Value", color="blue")
plt.legend(loc="upper right")
plt.ylim(-2, 2)
plt.show()

### 4.3.2 LSTM

* RNN모델은 입력시퀀스(층의 깊이)가 길어질 수록 앞쪽의 데이터가 뒤로 잘 전달되지 않아 학습능력이 저하
* 이런 문제를 해결하기 위해 RNN을 변형한 `LSTM Long Short Term Memory`를 개발

><img src="./images/0605.LSTM_03.png" width="400" height="300" />

### 4.3.3 양방향 LSTM

* RNN과 LSTM은 구조상 데이터가 입력된 순으로 처리되기 때문에 이전 시점의 정보만 활용가능하다는 단점이 있다
* 따라서, 문장이 길어질 수록 성능이 저하될 수 밖에 없다.
* 이런 문제를 해결하기 위해서 양방향LSTM(Bidirectional LSTM)모델이 개발 되었다.
* Bi-LSTM은 기존계층에 `역방향처리를 위한 LSTM계층을 하나 더 추가해서 양방향으로 문장의 패턴을 분석`
* 입력문장을 양방에서 처리하기 때문에 시퀀스의 길이가 길어진다 해도 `정보손실없이 처리가 가능`하다.
><img src="./images/0605.LSTM_02.png" width="400" height="300" />

### 4.3.4 개체명인식

* 임의의 문장에서 각 개체의 유형을 인식하는 것을 `개체명인식 Named Entity Recognization`이라고 한다.
* NER dlfks `문장에 포함된 단어가 인물, 조직, 장속등을 의미하는 단인지를 인식하는 것`을 말한다.
* 딥러닝모델이나 확률모델등을 이용해서 문자내에서 개체명을 인식하는 것을 `개체명인식기`라고 한다.
* 문장을 정확하게 인힉하기 위해서는 `반드시 처리해야 하는 전처리과정`이다.
* 개체명사전구축이 필요하다. 신조어, 사전미포함단어등의 처리는 불가능하기 때문에 사람이 직접
* 사전을 구축해야 되기 때문에 비용, 시간등이 많이 필요하게 된다.
* 개체명 인식모델을 만들기 위해서는 `BIO(Beginning Inside Outside)표기법`을 알아야 한다.
  - B : 개체명이 시작되는 단어에 `B-개체명`으로 태그
  - I : B-개체명과 연결되는 단어일 때는 `I-개체명`으로 태그
  - O : B,I제외한 모든 토큰에 태그
  >$$BIO표기예제$$
  >* 오늘부터 홍 길동은 삼성전자에 근무합니다.
  >* 오늘:B-Date  홍:B-Person   삼성:B-Company    근무:O
  >* 부터:O       길동:I-Person 전자:I-Company    합니다:O
  >*              은:O           에:O             .:O

* 개체명인식모델을 학습시키기 위해서는 `토큰별로 BIO태그가 달린 데이터셋이 필요`하다.
* 한글인 경우에는 BIO태그사전을 구하는 것이 어렵지만 `국립국어원 언어정보나눔센터에서 말뭉치를 공개`했다.
  - HLCT2016에서 제공하는 말뭉치를 수정한 KoreanNERCorpus:
  - 다운로드 : http://github.com/machinereading/KoreanNERCorpus
  - 훈련용데이터셋 : train.txt
    - `;로 시작하는 문장라인은 원본문장`
    - `$로 시작하는 문장라인은 해당 문장에서 NER처리된 결과`를 의미
    - 그 다음 라인 부터는 토큰번호, 단어토큰, 품사태그, BIO태그로 구성된 열
    - 강의에서는 `단어토큰과 BIO태그정보만 학습데이터셋으로 사용`
   
* NER인식에 필요한 시퀀스를 관리하는 라이브러리를 추가
  - `pip install seqeval`
  - https://pypi.org/project/seqeval
  - seqeval : Sequence labeling EVALuation 라이브러리

In [ ]:
!pip install seqeval
!pip show seqeval

In [1]:
# 양방향LSTM
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, TimeDistributed

In [2]:
# 1. 학습파일로딩
def read_file(filename):
    sents = []
    with open(filename, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx+1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx-1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents
                
# 학습용말뭉치데이터셋 로딩
corpus = read_file('./data/chatbot/train.txt')
print(type(corpus), len(corpus))
print(corpus[:1])

<class 'list'> 3555
[[('1', '한편', 'NNG', 'O'), ('1', ',', 'SP', 'O'), ('2', 'AFC', 'SL', 'O'), ('2', '챔피언스', 'NNG', 'O'), ('2', '리그', 'NNG', 'O'), ('3', 'E', 'SL', 'B_OG'), ('3', '조', 'NNG', 'I'), ('3', '에', 'JKB', 'O'), ('4', '속하', 'VV', 'O'), ('4', 'ㄴ', 'ETM', 'O'), ('5', '포항', 'NNP', 'O'), ('6', '역시', 'MAJ', 'O'), ('7', '대회', 'NNG', 'O'), ('8', '8강', 'NNG', 'O'), ('9', '진출', 'NNG', 'O'), ('9', '이', 'JKS', 'O'), ('10', '불투명', 'NNG', 'O'), ('10', '하', 'VV', 'O'), ('10', '다', 'EC', 'O'), ('11', '.', 'SF', 'O')]]


In [3]:
# 2. 학습용데이터셋생성 - corpus에서 단어와 BIO태그만 학습용데이터셋으로 사용
# 원본문장 : '; 한편, AFC챔피언스리그 E조에 속한 포항 역시 대회 8강 진출이 불투명하다 .'
# 1) 0번째 : 원본문장에서 분리된 단어 토큰들을 sentences에 저장
# 2) sentences의 단어 시퀀스에 해당하는 BIO태그 정보들을 tags에 저장
# 3) E조 : E-> B_OG, 조 -> I
# 4) 단어시퀀스의 평균길이(34.039)를 기준으로 시퀀스패딩크기를 결정
sentences, tags = [], []
for t in corpus:
    tagged_sentences = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentences.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    sentences.append(sentence)
    tags.append(bio_tag)

print(f'샘플데이터셋의 크기 = {len(sentences)}')
print(f'0번째 샘플문장의 시퀀스= {sentences[0]}')
print(f'0번째 샘플문장의 BIO 태그 = {tags[0]}')
print(f'샘플문장의 최대길이 = {max(len(l) for l in sentences)}')
print(f'샘플문장의 평균길이 = {sum(map(len, sentences)) / len(sentences)}')

샘플데이터셋의 크기 = 3555
0번째 샘플문장의 시퀀스= ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플문장의 BIO 태그 = ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플문장의 최대길이 = 168
샘플문장의 평균길이 = 34.03909985935302


In [4]:
# 3. 토크나이저정의
# 생성된 학습데이터셋에서 단어시퀀스와 태그시퀀스를 사전으로 등록하기 위해서
# 토크나이저를 정의한 후에 fit_on_texts()함수를 호출
# OOV : Out Of Vocabulary의 약자로 단어사전에 포함되지 않은 단어를 의미

# 1) 단어사전의 첫 번째 인덱스 토큰값을 OOV로 설정 및 토크나이징
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV')
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그정보는 소문자로 변환하지 않겠다.
tag_tokenizer.fit_on_texts(tags)
print(sent_tokenizer)
print(tag_tokenizer)

In [5]:
print(dir(tag_tokenizer))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api_export_path', '_api_export_symbol_id', 'analyzer', 'char_level', 'document_count', 'filters', 'fit_on_sequences', 'fit_on_texts', 'get_config', 'index_docs', 'index_word', 'lower', 'num_words', 'oov_token', 'sequences_to_matrix', 'sequences_to_texts', 'sequences_to_texts_generator', 'split', 'texts_to_matrix', 'texts_to_sequences', 'texts_to_sequences_generator', 'to_json', 'word_counts', 'word_docs', 'word_index']


In [6]:
print(sent_tokenizer.word_counts)
print(sent_tokenizer.word_docs)
print(sent_tokenizer.oov_token)

OrderedDict([('한편', 63), (',', 1650), ('afc', 2), ('챔피언스', 15), ('리그', 69), ('e', 14), ('조', 32), ('에', 1963), ('속하', 4), ('ㄴ', 2550), ('포항', 8), ('역시', 22), ('대회', 79), ('8강', 9), ('진출', 31), ('이', 3163), ('불투명', 1), ('하', 4222), ('다', 2517), ('.', 3629), ('2003', 6), ('년', 386), ('6', 170), ('월', 349), ('14', 51), ('일', 855), ('사직', 4), ('두산', 30), ('전', 199), ('이후', 57), ('박명환', 2), ('에게', 137), ('당하', 26), ('았', 1359), ('던', 204), ('10', 191), ('연패', 18), ('사슬', 1), ('을', 2713), ('거의', 8), ('5', 255), ('만', 343), ('끊', 3), ('는', 2560), ('의미', 14), ('있', 875), ('승리', 44), ('었', 1071), ('ap', 3), ('통신', 25), ('은', 1482), ('8', 130), ('(', 1084), ('이하', 39), ('한국', 168), ('시간', 85), (')', 1081), ('올라주원', 1), ('유잉', 1), ('비롯', 26), ('아', 1012), ('애드리언', 1), ('댄틀리', 1), ('팻', 1), ('라일리', 1), ('감독', 117), ('캐시', 2), ('러시', 1), ('tv', 39), ('해설가', 1), ('딕', 1), ('바이텔', 1), ('디트로이트', 5), ('피스톤스', 1), ('의', 1972), ('구단주', 2), ('윌리엄', 2), ('데이비드슨', 1), ('등', 455), ('2008', 63), ('명예', 12), (

In [7]:
# 2) 생성된 사전리스트(sent_tokenizer)를 이용해서 단어사전과 태그사전의 크기설정
vocab_size = len(sent_tokenizer.word_index)+1
tag_size = len(tag_tokenizer.word_index)+1
print(f'단어사전의 크기 = {vocab_size}')
print(f'BIO태그 단어사전의 크기 = {tag_size}')

단어사전의 크기 = 13834
BIO태그 단어사전의 크기 = 8


In [8]:
# 3) 학습용단어 시퀀스 생성
# sent_tokenizer 사전데이터를 시퀀스번호형태로 인코딩처리
X = sent_tokenizer.texts_to_sequences(sentences)
y = tag_tokenizer.texts_to_sequences(tags)
print(X[0], y[0])
# y_train의 1 = O, 3 = O_BG, 2 = I

[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3] [1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [9]:
# 4) Index of Word, Index of NER를 정의
index_to_word = sent_tokenizer.index_word # 단어시퀀인덱스(183)를 단어(한편)로 변환
index_to_ner = tag_tokenizer.index_word   # 태그시퀀스인덱스(3)를 개체명(B_OG)으로 변환
print(index_to_ner)
print(type(index_to_ner), index_to_ner[1], index_to_ner[3])
index_to_ner[0] = 'PAD'

{1: 'O', 2: 'I', 3: 'B_OG', 4: 'B_PS', 5: 'B_DT', 6: 'B_LC', 7: 'B_TI'}
<class 'dict'> O B_OG


In [10]:
# 5) 시퀀스패딩처리
# 개체명인식모델의 입출력벡터크기를 동일하게 설정하기 위해 시퀀스패딩작업을 진행
# max_len값은 단어시퀀의 평균길이(34.039)보다 넉넉하게 40으로 정의
max_len =40
X_padded = preprocessing.sequence.pad_sequences(X, padding='post', maxlen=max_len)
y_padded = preprocessing.sequence.pad_sequences(y, padding='post', maxlen=max_len)
print(X_padded[:1])
print(y_padded[:1])

[[ 183   11 4276  884  162  931  402   10 2608    7 1516  608  145 1361
   414    4 6347    2    8    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]
[[1 1 1 1 1 3 2 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0]]


In [11]:
# 6) 학습용 vs 검증용 = 8:2
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_padded, test_size=.2, random_state=0)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(2844, 40) (2844, 40) (711, 40) (711, 40)


In [12]:
# 7) 출력데이터를 원핫인코딩처리
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size) 
# print(y_train[:1])

In [13]:
# 8) 데이터전처리(정제)한 결과
print(f'단어사전의 크기 = {vocab_size}')
print(f'BIO태그 단어사전의 크기 = {tag_size}')
print(f'학습용 샘플 데이터의 시퀀스 크기 = {X_train.shape}')
print(f'학습용 샘플 데이터의 레이블 크기 = {y_train.shape}')
print(f'검증용 샘플 데이터의 시퀀스 크기 = {X_test.shape}')
print(f'검증용 샘플 데이터의 레이블 크기 = {y_test.shape}')

단어사전의 크기 = 13834
BIO태그 단어사전의 크기 = 8
학습용 샘플 데이터의 시퀀스 크기 = (2844, 40)
학습용 샘플 데이터의 레이블 크기 = (2844, 40, 8)
검증용 샘플 데이터의 시퀀스 크기 = (711, 40)
검증용 샘플 데이터의 레이블 크기 = (711, 40, 8)


In [14]:
# 4. Bi-LSTM모델정의
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Embedding, Dense, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam

In [21]:
# 1) 개체명인식모젤을 순차모델방식으로 정의
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.5, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 20s 220ms/step - accuracy: 0.5898 - loss: 1.0027
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 232ms/step - accuracy: 0.6424 - loss: 0.3337
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 231ms/step - accuracy: 0.6729 - loss: 0.2021
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 217ms/step - accuracy: 0.6784 - loss: 0.1661
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 219ms/step - accuracy: 0.6818 - loss: 0.1304
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 239ms/step - accuracy: 0.6923 - loss: 0.1053
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 212ms/step - accuracy: 0.6975 - loss: 0.0814
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 221ms/step - accuracy: 0.6993 - loss: 0.0626
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 254ms/step - accuracy: 0.7091 - loss: 0.0584
Epoch 10/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 258ms/step - accuracy: 0.6923 - loss: 0.0531
Epoch 11/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 228ms/step - accuracy: 0.7117 - loss: 0.0446
Epoch 12/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 

In [23]:
# 2) 모델평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'평가결과(손실율)= {loss:.3f}')
print(f'평가결과(정확도)= {accuracy:.3f}')

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.6653 - loss: 0.4257
평가결과(손실율)= 0.449
평가결과(정확도)= 0.662


##### 결과분석
* 정확도가 66%정도이지만 개체명인식에서 사용되는 평가는 `f1-score로 성능평가`를 해야 한다.
* f1-score를 계산하기 위해서는 `정밀도와 재현율을 사용`해야 한다.
  - 정확도(Accuracy) - 실제 정답과 얼마나 유사한지의 척도
  - 정밀도(Precision) - `정밀도가 높으면 결과값이 일정하게 분포`되어 있는지의 척도
  - 재현율(Recall) : `실제 정답을 예측모델이 정답으로 예측한 비율`을 나타내는 척도
  - 공식
    >$$f1score = 2 * \frac{정밀도 * 재현율}{정밀도 + 재현율}$$

In [43]:
# 시퀀스를 NER로 변환함수
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(temp)
    return result

In [44]:
# 테스트데이터셋의 NER예측
y_predicted = model.predict(X_test)
y_predicted[:1]

pred_tags = sequences_to_tag(y_predicted)  # 예측 개체명인식명의 태그
test_tags = sequences_to_tag(y_test)       # 실제 개체명인식명의 태그

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [47]:
# f1-score 계산
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print(f'f1-score = {f1_score(test_tags, pred_tags):.3f}')

              precision    recall  f1-score   support

           _       0.65      0.63      0.64       657
         _DT       0.91      0.92      0.92       335
         _LC       0.69      0.59      0.64       312
         _OG       0.73      0.55      0.63       481
         _PS       0.71      0.47      0.57       374
         _TI       0.92      0.86      0.89        66

   micro avg       0.74      0.63      0.68      2225
   macro avg       0.77      0.67      0.71      2225
weighted avg       0.73      0.63      0.67      2225

f1-score = 0.679


In [55]:
# 문장예측
word_to_index = sent_tokenizer.word_index
print(type(word_to_index))

new_sentence = '삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.'.split()
new_sentence
new_list = []

for w in new_sentence:
    try:
        new_list.append(word_to_index.get(w, 1))
    except KeyError:
        # 모르는 단어의 경우 OOV
        new_list.append(word_index['OOV'])
        
print(f'새로운 문장의 단어 토큰 = {new_sentence}')
print(f'새로운 문장의 단어 시퀀스 = {new_list}')

<class 'dict'>
새로운 문장의 단어 토큰 = ['삼성전자', '출시', '스마트폰', '오늘', '애플', '도전장', '내밀다.']
새로운 문장의 단어 시퀀스 = [531, 307, 1476, 286, 1507, 6766, 1]


In [69]:
# NER예측
new_padded_seqs = preprocessing.sequence.pad_sequences([new_list]
                                                       , padding='post'
                                                       , value=0
                                                       , maxlen=max_len)
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=1) # 예측된 NER의 인덱스값을 추출
print('{:10} {:5}'.format("단어", "예측된 NER"))
print('-'*50)
for w, pred in zip(new_sentence, p[0]):
    print('{:10} {:5}'.format(w, index_to_ner[pred]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
단어         예측된 NER
--------------------------------------------------
삼성전자       B_TI 
출시         O    
스마트폰       B_LC 
오늘         PAD  
애플         B_TI 
도전장        B_OG 
내밀다.       B_TI 


In [70]:
삼성전자       B_OG 
출시           I     
스마트폰       O
오늘           B_DT  
애플           B_OG 
도전장         I 
내밀다.        I

SyntaxError: invalid syntax (2395243856.py, line 1)